In [1]:
import os

from bs4 import BeautifulSoup as bs, NavigableString, Comment
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [7]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [1]:
driver.get('http://localhost:3000/antd')

NameError: name 'driver' is not defined

In [ ]:
class Tag:
    def __init__(self, selenium_web_element, parent):
        self.parent = parent
        self.xpath_counter = {}
        self.children = []
        self.styles = {}
        self.level = None
        self.tag_name = None
        self.xpath = None
    
    
    def _initialize_from_selenium_web_element(self, selenium_web_element, parent):
        self.tag_name = selenium_web_element.tag_name
        self.level = parent.get_level() + 1
        # These next two lines are correlated and can generate bug (probably in parallel situations).
        # Refactor them.
        self.xpath = parent.generate_new_xpath(self.tag_name)
        parent.add_child(self)
        # self.styles = selenium_web_element. (getComputedStyle)
    
    
    def generate_new_xpath(self, tag_name):
        if not tag_name in self.xpath_counter:
            self.xpath_counter[tag_name] = 0
        self.xpath_counter[tag_name] += 1
        return '{0}/{1}[{2}]'.format(self.xpath, tag_name, self.xpath_counter[tag_name])
    
    
    def add_child(self, child):
        self.children.append(child)
    
    
    def get_level(self):
        return self.level or 0

In [3]:
driver.get('https://www.aircanada.com/us/en/aco/home.html')

In [4]:
doc = bs(driver.find_element(By.TAG_NAME, 'html').get_attribute('innerHTML'))

In [6]:
print(doc.prettify())

<html>
 <head class="at-element-marker">
  <meta charset="utf-8"/>
  <title>
   Air Canada -  Official website
  </title>
  <base href="/"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="max-age=0" http-equiv="cache-control"/>
  <meta content="no-store" http-equiv="cache-control"/>
  <meta content="0" http-equiv="expires"/>
  <meta content="Tue, 01 Jan 1980 1:00:00 GMT" http-equiv="expires"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <link as="font" crossorigin="use-credentials" href="/achome/assets/font/open-sans/OpenSans.woff2" rel="dns-prefetch" type="font/woff2"/>
  <link as="font" crossorigin="use-credentials" href="/achome/assets/font/icomoon/icomoon.woff" rel="dns-prefetch" type="font/woff2"/>
  <style>
   @font-face {
				font-family: 'Open Sans';
				src: url('/achome/assets/font/open-sans/OpenSans.woff2') format("woff2");
				font-weight: 300;
				font-display: swap;
			}

			@font-face {
				font-family: 'Open Sans';
				src: url('/achome

# Delete Unnecessary Tags

In [12]:
UNNECESSARY_TAGS = [
    'style', 'script', 'noscript', 'link'
]

In [13]:
root = doc.html
root.attrs['xpath'] = '/html'
traverse_queue = [root]

while len(traverse_queue) > 0:
    counter = 1
    node = traverse_queue[0]
    for child in node.children:
        if isinstance(child, Comment):
            child.extract()
            continue
        elif isinstance(child, NavigableString):
            continue
        elif child.name in UNNECESSARY_TAGS:
            counter += 1
        else:
            traverse_queue.append(child)
            child.attrs['xpath'] = '{0}/*[{1}]'.format(node.attrs['xpath'], counter)
            counter += 1
    traverse_queue = traverse_queue[1:]

In [14]:
for tag_type in UNNECESSARY_TAGS:
    tags = doc.find_all(tag_type)
    for tag in tags:
        tag.decompose()

In [15]:
print(doc.prettify())

<html xpath="/html">
 <head class="at-element-marker" xpath="/html/*[1]">
  <meta charset="utf-8" xpath="/html/*[1]/*[1]"/>
  <title xpath="/html/*[1]/*[2]">
   Air Canada -  Official website
  </title>
  <base href="/" xpath="/html/*[1]/*[3]"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible" xpath="/html/*[1]/*[4]"/>
  <meta content="max-age=0" http-equiv="cache-control" xpath="/html/*[1]/*[5]"/>
  <meta content="no-store" http-equiv="cache-control" xpath="/html/*[1]/*[6]"/>
  <meta content="0" http-equiv="expires" xpath="/html/*[1]/*[7]"/>
  <meta content="Tue, 01 Jan 1980 1:00:00 GMT" http-equiv="expires" xpath="/html/*[1]/*[8]"/>
  <meta content="no-cache" http-equiv="pragma" xpath="/html/*[1]/*[9]"/>
  <meta content="width=device-width, initial-scale=1" name="viewport" xpath="/html/*[1]/*[13]"/>
  <meta content="#fffffe" media="(prefers-color-scheme: light)" name="theme-color" xpath="/html/*[1]/*[14]"/>
  <meta content="#fffffe" media="(prefers-color-scheme: dark)" name="th

In [20]:
driver.find_element(By.XPATH, '/html/*[2]/*[77]').get_attribute('innerHTML')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/*[2]/*[77]"}
  (Session info: chrome=109.0.5414.119)
Stacktrace:
0   chromedriver                        0x0000000109871fa8 chromedriver + 4886440
1   chromedriver                        0x00000001097ef643 chromedriver + 4351555
2   chromedriver                        0x000000010943db27 chromedriver + 477991
3   chromedriver                        0x000000010948276c chromedriver + 759660
4   chromedriver                        0x00000001094829c1 chromedriver + 760257
5   chromedriver                        0x00000001094c6204 chromedriver + 1036804
6   chromedriver                        0x00000001094a85fd chromedriver + 914941
7   chromedriver                        0x00000001094c35fe chromedriver + 1025534
8   chromedriver                        0x00000001094a83a3 chromedriver + 914339
9   chromedriver                        0x000000010947257f chromedriver + 693631
10  chrom

In [46]:
for obj in doc.descendants:
    print(obj.name, isinstance(obj, Comment), obj.parent.name, type(obj))
    if obj.name is None:
        print(obj.string)

html False [document] <class 'bs4.element.Tag'>
head False html <class 'bs4.element.Tag'>
None False head <class 'bs4.element.NavigableString'>


meta False head <class 'bs4.element.Tag'>
None False head <class 'bs4.element.NavigableString'>


title False head <class 'bs4.element.Tag'>
None False title <class 'bs4.element.NavigableString'>
Air Canada -  Official website
None False head <class 'bs4.element.NavigableString'>


base False head <class 'bs4.element.Tag'>
None False head <class 'bs4.element.NavigableString'>


meta False head <class 'bs4.element.Tag'>
None False head <class 'bs4.element.NavigableString'>


meta False head <class 'bs4.element.Tag'>
None False head <class 'bs4.element.NavigableString'>


meta False head <class 'bs4.element.Tag'>
None False head <class 'bs4.element.NavigableString'>


meta False head <class 'bs4.element.Tag'>
None False head <class 'bs4.element.NavigableString'>


meta False head <class 'bs4.element.Tag'>
None False head <class 'bs4.element.Nav

h2 False div <class 'bs4.element.Tag'>
None False h2 <class 'bs4.element.NavigableString'>
 Top of page banner 
ul False div <class 'bs4.element.Tag'>
li False ul <class 'bs4.element.Tag'>
div False li <class 'bs4.element.Tag'>
div False div <class 'bs4.element.Tag'>
picture False div <class 'bs4.element.Tag'>
source False picture <class 'bs4.element.Tag'>
source False source <class 'bs4.element.Tag'>
source False source <class 'bs4.element.Tag'>
source False source <class 'bs4.element.Tag'>
None True source <class 'bs4.element.Comment'>
 
img False source <class 'bs4.element.Tag'>
None True source <class 'bs4.element.Comment'>
 
div False div <class 'bs4.element.Tag'>
div False div <class 'bs4.element.Tag'>
span False div <class 'bs4.element.Tag'>
None False span <class 'bs4.element.NavigableString'>
NEW OFFER
div False div <class 'bs4.element.Tag'>
h3 False div <class 'bs4.element.Tag'>
None False h3 <class 'bs4.element.NavigableString'>
Earn up to 100K pts, worth $1,250 in travel va

None True div <class 'bs4.element.Comment'>
 
abc-form-element-underline False div <class 'bs4.element.Tag'>
div False abc-form-element-underline <class 'bs4.element.Tag'>
abc-subscript False div <class 'bs4.element.Tag'>
div False abc-subscript <class 'bs4.element.Tag'>
None True div <class 'bs4.element.Comment'>
 
None True div <class 'bs4.element.Comment'>
 
None True div <class 'bs4.element.Comment'>
 
None True abc-subscript <class 'bs4.element.Comment'>
 
None True div <class 'bs4.element.Comment'>
 
None True div <class 'bs4.element.Comment'>
 
div False div <class 'bs4.element.Tag'>
None True div <class 'bs4.element.Comment'>
 
div False div <class 'bs4.element.Tag'>
None False div <class 'bs4.element.NavigableString'>
 Enter the three first characters of the city, airport, province, state or country/region you are flying to, then use your arrow keys to move up or down the list of airports provided to make your selection. 
None True div <class 'bs4.element.Comment'>
 
None True

div False abc-page-loader <class 'bs4.element.Tag'>
div False div <class 'bs4.element.Tag'>
h2 False div <class 'bs4.element.Tag'>
span False h2 <class 'bs4.element.Tag'>
None False span <class 'bs4.element.NavigableString'>
Please wait while content is loading.
div False div <class 'bs4.element.Tag'>
abc-inset-loader False div <class 'bs4.element.Tag'>
div False abc-inset-loader <class 'bs4.element.Tag'>
div False div <class 'bs4.element.Tag'>
h2 False div <class 'bs4.element.Tag'>
span False h2 <class 'bs4.element.Tag'>
None False span <class 'bs4.element.NavigableString'>
Please wait while content is loading.
None True h2 <class 'bs4.element.Comment'>
 
None True div <class 'bs4.element.Comment'>
 
div False div <class 'bs4.element.Tag'>
div False div <class 'bs4.element.Tag'>
svg False div <class 'bs4.element.Tag'>
g False svg <class 'bs4.element.Tag'>
circle False g <class 'bs4.element.Tag'>
g False g <class 'bs4.element.Tag'>
path False g <class 'bs4.element.Tag'>
path False g <c

In [7]:
# remove unnecessary tags



In [10]:
print(doc.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   Air Canada -  Official website
  </title>
  <base href="/"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="max-age=0" http-equiv="cache-control"/>
  <meta content="no-store" http-equiv="cache-control"/>
  <meta content="0" http-equiv="expires"/>
  <meta content="Tue, 01 Jan 1980 1:00:00 GMT" http-equiv="expires"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="#fffffe" media="(prefers-color-scheme: light)" name="theme-color"/>
  <meta content="#fffffe" media="(prefers-color-scheme: dark)" name="theme-color"/>
  <!-- generics -->
  <!-- Android -->
  <!-- iOS -->
  <!-- Windows 8 IE 10-->
  <meta content="#000000" name="msapplication-TileColor"/>
  <meta content="/achome/assets/img/favicon/mstile-144x144.png" name="msapplication-TileImage"/>
 </head>
 <body>
  <iframe role="presentation" src="javascript:void(0)" styl

In [26]:
for child in doc.html.children:
    print(child.name)

head
body


In [22]:
doc.body.find_all('form')[0]

<form action="/us/en/aco/home/search-results.html?q=" autocomplete class="ngx-ac-search-form ng-star-inserted" method="get" role="search"><abc-input class="ngx-ac-search-source-header abc-form-element"><abc-form-element-container><div class="abc-form-element-container abc-form-element-material-appearance-borderless abc-form-element-material-appearance-condensed"><div class="abc-form-element-wrapper abc-form-element-material-wrapper abc-form-element-input-wrapper ng-star-inserted"><div class="abc-form-element-flex"><!-- --><div class="abc-form-element-infix"><!-- --><div class="abc-form-element-main-container"><!-- --><!-- --><input aria-disabled="false" aria-invalid="false" aria-label="Search site" autocomplete="off" autocorrect="off" class="abc-form-element-input abc-form-element-main text-transform-none ng-untouched ng-pristine ng-valid ng-star-inserted" id="acSiteSearchInput" name="q" placeholder="" spellcheck="false" type="search"/><!-- --></div><abc-form-element-label class="ng-st

In [17]:
doc.find_all('form')[0].attrs

{'role': 'search',
 'method': 'get',
 'autocomplete': 'off',
 'class': ['ngx-ac-search-form', 'ng-star-inserted'],
 'action': '/us/en/aco/home/search-results.html?q='}

In [13]:
driver.find_element(By.XPATH, '/html')

<selenium.webdriver.remote.webelement.WebElement (session="b87b258179c1e388763caeb0613a0892", element="9ae43bc6-7378-4ff9-a398-5a2215d0224d")>